In [138]:
import copy, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

file_name = 'wisconsin'
csv_path = './team_adv_stats_avg/' + str(file_name) + '.csv'

In [139]:
df = pd.read_csv('./team_stats/' + str(file_name) + '_adv_2023.csv')

df.drop(['Tm', 'Opp.1', 'W/L'], axis=1, inplace=True)

df = df.loc[:, ~df.columns.str.startswith('Unnamed')]

df.columns

Index(['Date', 'Opp', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'TRB%',
       'AST%', 'STL%', 'BLK%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA', 'eFG%.1',
       'TOV%.1', 'DRB%', 'FT/FGA.1'],
      dtype='object')

In [140]:
column_rename_mapping = {'Date': 'Date', 'Opp': 'Opp', 'Tm': 'Tm', 'Opp.1': 'Opp_Score', 'ORtg': 'ORtg', 'DRtg': 'DRtg', 'Pace': 'Pace', 'FTr': 'FTr', '3PAr': '3PAr',
       'TS%': 'TS%', 'TRB%': 'TRB%', 'AST%': 'AST%', 'STL%': 'STL%', 'BLK%': 'BLK%', 'eFG%': 'eFG%', 'TOV%': 'TOV%', 'ORB%': 'ORB%', 'FT/FGA': 'FT/FGA',
       'eFG%.1': 'Opp_eFG%.1', 'TOV%.1': 'Opp_TOV%', 'DRB%': 'DRB%', 'FT/FGA.1': 'Opp_FT/FGA'}

df.rename(columns=column_rename_mapping, inplace=True)

df

,Date,Opp,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,Opp_eFG%.1,Opp_TOV%,DRB%,Opp_FT/FGA
0,2022-11-07,South Dakota,121.4,84.3,69.5,0.418,0.473,0.645,53.7,50.0,11.4,9.5,0.618,13.2,19.4,0.309,0.421,13.3,83.3,0.193
1,2022-11-11,Stanford,88.2,73.5,68.2,0.305,0.339,0.444,45.2,40.9,16.2,12.9,0.415,10.6,20.0,0.186,0.372,21.6,77.8,0.319
2,2022-11-15,Green Bay,90.3,72.6,62.0,0.358,0.358,0.451,64.6,50.0,9.7,6.7,0.377,18.4,40.0,0.302,0.364,14.1,93.3,0.295
3,2022-11-23,Dayton,66.2,64.6,64.8,0.169,0.458,0.337,46.7,50.0,12.3,3.8,0.288,13.6,22.0,0.153,0.340,20.8,76.5,0.113
4,2022-11-24,Kansas,94.4,95.8,64.0,0.306,0.306,0.479,44.2,41.7,11.1,3.0,0.435,7.8,16.2,0.226,0.460,15.3,70.0,0.194
5,2022-11-25,USC,101.6,93.7,63.3,0.143,0.429,0.535,45.9,52.0,9.5,3.0,0.536,10.5,13.8,0.071,0.509,18.1,75.0,0.036
6,2022-11-29,Wake Forest,108.7,113.0,69.1,0.321,0.500,0.581,42.3,58.3,10.1,3.3,0.536,14.6,21.4,0.268,0.611,19.6,66.7,0.222
7,2022-12-03,Marquette,115.9,111.6,61.2,0.220,0.458,0.614,54.5,70.0,13.0,3.0,0.610,16.6,30.8,0.136,0.594,16.1,75.9,0.264
8,2022-12-06,Maryland,95.5,88.1,66.8,0.574,0.447,0.535,43.1,70.0,10.4,9.7,0.500,11.8,3.3,0.362,0.473,18.7,77.1,0.127
9,2022-12-11,Iowa,100.0,96.2,69.0,0.169,0.338,0.555,45.2,51.5,15.4,2.7,0.554,13.5,11.4,0.092,0.477,17.2,76.3,0.219


In [141]:
averages_df = df.copy()

for i in range(5, len(df)):
    # Calculate the mean of the previous 5 rows
    averages_df.loc[i, df.columns[2:]] = df.loc[i-5:i-1, df.columns[1:]].mean()

averages_df

C:\Users\lakea\AppData\Local\Temp\ipykernel_18060\725090269.py:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  averages_df.loc[i, df.columns[2:]] = df.loc[i-5:i-1, df.columns[1:]].mean()


,Date,Opp,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,Opp_eFG%.1,Opp_TOV%,DRB%,Opp_FT/FGA
0,2022-11-07,South Dakota,121.400,84.300,69.500,0.41800,0.47300,0.64500,53.700,50.000,11.400,9.500,0.61800,13.200,19.400,0.30900,0.42100,13.300,83.300,0.19300
1,2022-11-11,Stanford,88.200,73.500,68.200,0.30500,0.33900,0.44400,45.200,40.900,16.200,12.900,0.41500,10.600,20.000,0.18600,0.37200,21.600,77.800,0.31900
2,2022-11-15,Green Bay,90.300,72.600,62.000,0.35800,0.35800,0.45100,64.600,50.000,9.700,6.700,0.37700,18.400,40.000,0.30200,0.36400,14.100,93.300,0.29500
3,2022-11-23,Dayton,66.200,64.600,64.800,0.16900,0.45800,0.33700,46.700,50.000,12.300,3.800,0.28800,13.600,22.000,0.15300,0.34000,20.800,76.500,0.11300
4,2022-11-24,Kansas,94.400,95.800,64.000,0.30600,0.30600,0.47900,44.200,41.700,11.100,3.000,0.43500,7.800,16.200,0.22600,0.46000,15.300,70.000,0.19400
5,2022-11-25,USC,92.100,78.160,65.700,0.31120,0.38680,0.47120,50.880,46.520,12.140,7.180,0.42660,12.720,23.520,0.23520,0.39140,17.020,80.180,0.22280
6,2022-11-29,Wake Forest,88.140,80.040,64.460,0.25620,0.37800,0.44920,49.320,46.920,11.760,5.880,0.41020,12.180,22.400,0.18760,0.40900,17.980,78.520,0.19140
7,2022-12-03,Marquette,92.240,87.940,64.640,0.25940,0.41020,0.47660,48.740,50.400,10.540,3.960,0.43440,12.980,22.680,0.20400,0.45680,17.580,76.300,0.17200
8,2022-12-06,Maryland,97.360,95.740,64.480,0.23180,0.43020,0.50920,46.720,54.400,11.200,3.220,0.48100,12.620,20.840,0.17080,0.50280,17.980,72.820,0.16580
9,2022-12-11,Iowa,103.220,100.440,64.880,0.31280,0.42800,0.54880,46.000,58.400,10.820,4.400,0.52340,12.260,17.100,0.21260,0.52940,17.560,72.940,0.16860


In [142]:
averages_df.to_csv(csv_path, index=False)